機械学習アルゴリズムが処理しやすいよう下準備する
============================================

In [2]:
import numpy as np
import pandas as pd
import pathlib

train_data_path = pathlib.Path("../input/spaceship-titanic/train.csv")
train_data = pd.read_csv(
    train_data_path,
    dtype={
        "PassengerId": "string",
        "HomePlanet": "string",
        "Cabin": "string",
        "Destination": "string",
    },
)

## 不要な属性を削除する

いままでデータの可視化を進めてきました．これに基づいて，不要だとわかる特徴量を削除してみましょう．

後でテストデータに対しても適用しないといけないので，再利用を意識して scikit-learn のパイプラインとして削除処理を作ります．

In [37]:
from sklearn.compose import ColumnTransformer

drop_attribs = ["Name", "VIP", "FoodCourt", "ShoppingMall"]

pipeline = ColumnTransformer(
    [
        ("drop", "drop", drop_attribs),
    ],
    remainder="passthrough"
)

pipeline.set_output(transform="pandas")

pipeline.fit_transform(train_data)

,remainder__PassengerId,remainder__HomePlanet,remainder__CryoSleep,remainder__Cabin,remainder__Destination,remainder__Age,remainder__RoomService,remainder__Spa,remainder__VRDeck,remainder__Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,109.0,549.0,44.0,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,43.0,6715.0,49.0,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,0.0,3329.0,193.0,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,303.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,0.0,1643.0,74.0,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,0.0,0.0,0.0,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,0.0,1.0,0.0,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,0.0,353.0,3235.0,False


```{note}
`ColumnTransformer.fit_transform` の返り値の型は行列であって DataFrame ではないのですが，それだと列名が落ちてしまって見づらいので `set_output` を使って DataFrame を返すように変更しています．
```

ここでは `Name`, `VIP`, `FoodCourt`, `ShoppingMall` を削除することにしました．

* `Name` は人名なので，あまり関係がないだろうと判断しました．

* `VIP` は値に偏りがありすぎます．

* `FoodCourt` と `ShoppingMall` は，予測対象の列との相関が微妙で，今回は単純化のために考えないことにしました．

## 各列に下処理をする

残った特徴量についても，そのまま機械学習アルゴリズムに読ませることはできません．

アルゴリズムが処理しやすいように，扱いやすい形に変換します．

### HomePlanet

`HomePlanet` についてまず考えます．

* 201 行の欠損値を持ち，
* 3通りの値をとるカテゴリ属性
* `Europa` だと異次元に飛ばされやすく，`Earth` だと飛ばされにくい．`Mars` はほぼ無関係

ということがわかっています．

欠損値のない数値属性に変換しましょう．

まずは欠損値を処理します．

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class HomePlanetTransformer(BaseEstimator, TransformerMixin):
    def __init__(self) -> None:
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        pass

In [52]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

drop_attribs = ["Name", "VIP", "FoodCourt", "ShoppingMall"]

pipeline = ColumnTransformer(
    [
        ("drop", "drop", drop_attribs),

        # 欠損値を "Mars" という値に置き換える
        ("imp", SimpleImputer(strategy="constant", fill_value="Mars", missing_values=pd.NA), ["HomePlanet"])
    ],
    remainder="passthrough"
)

pipeline.set_output(transform="pandas")

pipeline.fit_transform(train_data).isnull().sum()

imp__HomePlanet             0
remainder__PassengerId      0
remainder__CryoSleep      217
remainder__Cabin          199
remainder__Destination    182
remainder__Age            179
remainder__RoomService    181
remainder__Spa            183
remainder__VRDeck         188
remainder__Transported      0
dtype: int64